In [3]:
## Welcome to the my Tennessee campaign finance internal database

In [4]:
import pandas as pd

In [5]:
# This data comes from the Tennessee Registry of Election Finance campaign: https://apps.tn.gov/tncamp/public/cesearch.htm
# I searched all donations made by each year, downloaded all the files and make a combined csv by year. This files are available in a seperate folder
# NOTE: Contributions in year often run Jan. 16 to Jan. 15, so the report year won't always match date

dfs = []

#This is a loop because I named all my files the same
#I added title report year as reference for fact checking later
#NOTE: This does not include the the first half 2024 year for workflow purpose
# On Jan. 30, 2025, this needs to be updated to (2009,2025)
# Have to put 2024, otherwise it won't read the 2023 file

for year in range(2009,2024):
    df = pd.read_csv(f'Original TN contributions {year}.csv', low_memory=False)
    df['Title Report Year'] = year
    df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
    dfs.append(df)

# This will need to be replace going forward 
df_2024 = pd.read_csv('Original TN Contributions 2024 thru July 25.csv',low_memory=False)
df_2024['Title Report Year'] = 2024
df_2024['Amount'] = pd.to_numeric(df_2024['Amount'], errors='coerce')

#Bringing in Bill Lee's inaugural files
#DATA CAVEAT: The inaugural reports for Gov. Bill Haslam are not available
df_Lee_2019 = pd.read_csv('Gov. Bill Lee 2019 Inauguaration contributions.csv',low_memory=False)
df_Lee_2019['Title Report Year'] = 2019
df_Lee_2019['Amount'] = pd.to_numeric(df_Lee_2019['Amount'], errors='coerce')

df_Lee_2023 = pd.read_csv('Gov. Bill Lee 2023 Inauguaration contributions.csv',low_memory=False)
df_Lee_2023['Title Report Year'] = 2023
df_Lee_2023['Amount'] = pd.to_numeric(df_Lee_2023['Amount'], errors='coerce')

#Adding the Lee inaguaration and 2024 file into the overall dataframe
dfs.append(df_2024)
dfs.append(df_Lee_2019)
dfs.append(df_Lee_2023)

#Combining all the dataframes into one file, this file is called the 2009 to most recent date available
df_tn_contributions_2009_to_June_30_2024 = pd.concat(dfs, ignore_index=True)

In [6]:
#Now I'm going to clean up the df_tn_contributions_2009_to_June_30_2024 before I start cross referencing it with other files

#I'm going to rename Recipient Name to original-name-of-recipient-of-donation 
df_tn_contributions_2009_to_June_30_2024 = df_tn_contributions_2009_to_June_30_2024.rename(columns={'Recipient Name': 'original-name-of-recipient-of-donation'})

#I'm going to rename Contributor Name to original-name-of-donor
df_tn_contributions_2009_to_June_30_2024 = df_tn_contributions_2009_to_June_30_2024.rename(columns={'Contributor Name': 'original-name-of-donor'})

#Lets make everything proper names instead of all caps

df_tn_contributions_2009_to_June_30_2024['Type'] = df_tn_contributions_2009_to_June_30_2024['Type'].str.title()
df_tn_contributions_2009_to_June_30_2024['Adj'] = df_tn_contributions_2009_to_June_30_2024['Adj'].str.title()
df_tn_contributions_2009_to_June_30_2024['Date'] = df_tn_contributions_2009_to_June_30_2024['Date'].str.title()
df_tn_contributions_2009_to_June_30_2024['Report Name'] = df_tn_contributions_2009_to_June_30_2024['Report Name'].str.title()
df_tn_contributions_2009_to_June_30_2024['original-name-of-recipient-of-donation'] = df_tn_contributions_2009_to_June_30_2024['original-name-of-recipient-of-donation'].str.title()
df_tn_contributions_2009_to_June_30_2024['original-name-of-donor'] = df_tn_contributions_2009_to_June_30_2024['original-name-of-donor'].str.title()
df_tn_contributions_2009_to_June_30_2024['Contributor Address'] = df_tn_contributions_2009_to_June_30_2024['Contributor Address'].str.title()
df_tn_contributions_2009_to_June_30_2024['Contributor Occupation'] = df_tn_contributions_2009_to_June_30_2024['Contributor Occupation'].str.title()
df_tn_contributions_2009_to_June_30_2024['Contributor Employer'] = df_tn_contributions_2009_to_June_30_2024['Contributor Employer'].str.title()

# Dealing with the amount column
df_tn_contributions_2009_to_June_30_2024['Amount'] = df_tn_contributions_2009_to_June_30_2024['Amount'].fillna(0)

# Ensure 'amount' is converted to integers if required
df_tn_contributions_2009_to_June_30_2024['Amount'] = df_tn_contributions_2009_to_June_30_2024['Amount'].astype(int)

In [7]:
#Need to make sure date is read as a date


#There are 2196 rows where no date is given, but I know the year of the report, therefor I'm going to change the date to Jan. 1 of that year, late I'll drop the date column

df_tn_contributions_2009_to_June_30_2024['Date'] = df_tn_contributions_2009_to_June_30_2024.apply(
    lambda row: pd.to_datetime(f"1-1-{row['Title Report Year']}") if pd.isnull(row['Date']) else row['Date'], axis=1
)
# Convert 'Date' column to datetime, handling errors by coercing to NaT
df_tn_contributions_2009_to_June_30_2024['Date'] = pd.to_datetime(df_tn_contributions_2009_to_June_30_2024['Date'])
df_tn_contributions_2009_to_June_30_2024['Date'] = df_tn_contributions_2009_to_June_30_2024['Date'].dt.strftime('%m/%d/%Y')

# I will get an error here, but I can ignore it and keep going

/var/folders/tt/pm7f52p57852vpw35wsk8mgr0000gn/T/ipykernel_10082/1651428919.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_tn_contributions_2009_to_June_30_2024['Date'] = pd.to_datetime(df_tn_contributions_2009_to_June_30_2024['Date'])


In [8]:
# Run data time again no error 
df_tn_contributions_2009_to_June_30_2024['Date'] = pd.to_datetime(df_tn_contributions_2009_to_June_30_2024['Date'])

In [9]:
## Remove dollar signs and commas, and handle parentheses
df_tn_contributions_2009_to_June_30_2024['Amount'] = df_tn_contributions_2009_to_June_30_2024['Amount'].replace('[\$,]', '', regex=True)

# Convert to numeric
df_tn_contributions_2009_to_June_30_2024['Amount'] = pd.to_numeric(df_tn_contributions_2009_to_June_30_2024['Amount'])

#For adjustment if its say Y it means it was returned, and therefor its basically a negative return
#First renaming it to adjustment because that's easier to understand
df_tn_contributions_2009_to_June_30_2024 = df_tn_contributions_2009_to_June_30_2024.rename(columns={'Adj': 'Adjustment'})
df_tn_contributions_2009_to_June_30_2024.loc[df_tn_contributions_2009_to_June_30_2024['Adjustment'] == 'Y', 'Amount'] = df_tn_contributions_2009_to_June_30_2024['Amount'] * -1
df_tn_contributions_2009_to_June_30_2024['Adjustment'] = df_tn_contributions_2009_to_June_30_2024['Adjustment'].replace({'N': '', 'Y': 'Money Returned'})

In [10]:
#Dropping null values
df_tn_contributions_2009_to_June_30_2024 = df_tn_contributions_2009_to_June_30_2024.dropna(subset=['original-name-of-donor'])


#Now at this point I'm saving this giant file in a dataframe called df_tn_contributions_2009_to_June_30_2024_raw

df_tn_contributions_2009_to_June_30_2024_raw = df_tn_contributions_2009_to_June_30_2024


In [11]:
#This is where it gets more difficult

#I'm reading in the giant file of candidates and creating dataframe for future reference that has distinct names with their history
df_tn_candidates_09to24_original = pd.read_csv('All people who ran for office 2009 to 2024.csv')

#Need to get each politicians name and their results for reference purposes for later
#This will also contain PACs and candidates linked to them
#Everything is merged on original name
#The new dataframe will be called df_tn_candidates_09to24

df_tn_candidates_09to24_original['Results'] = df_tn_candidates_09to24_original.apply(
    lambda row: f"{row['Year']} : {row['office sought']} - {row['General win/lost']}", axis=1)

df_tn_candidates_09to24 = df_tn_candidates_09to24_original.groupby('Original-name').agg({
    'Original-name': 'first',
    'Fixed-name': 'first',
    'Party': 'first',
    'City': 'first',
    'Results': lambda x: ' | '.join(x)
}).reset_index(drop=True)

In [12]:
#This file contains more specific names of contributors, mainly corporations and big family donors
df_behind_the_pacs = pd.read_csv('The company behind the PAC and mergers.csv')

In [13]:
#I want all the donations made to candidate, parties and PACs affilated with those
df_tn_contributions_2009_to_June_30_2024_merging_donor_recipient_names = pd.merge(df_tn_contributions_2009_to_June_30_2024, df_tn_candidates_09to24, left_on='original-name-of-recipient-of-donation', right_on='Original-name', how='left')

df_tn_contributions_2009_to_June_30_2024_merging_donor_recipient_names = pd.merge(df_tn_contributions_2009_to_June_30_2024_merging_donor_recipient_names, df_behind_the_pacs, left_on='original-name-of-donor', right_on='Original-donor-name', how='left')

#Creating a raw file of all merging, this services as save point
df_tn_contributions_2009_to_June_30_2024_merging_donor_recipient_names_raw = df_tn_contributions_2009_to_June_30_2024_merging_donor_recipient_names

In [14]:
#Now I'm reorganizing the files so that I have what I want

df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo = df_tn_contributions_2009_to_June_30_2024_merging_donor_recipient_names.rename(columns={
    'Fixed-name':'name-of-recipient-of-donation',
    'Company-affiliated-name' : 'name-of-donor',
    'Amount': 'donation-amount',
    'Date' : 'date-of-donation',
    'Title Report Year' : 'report-year',
    'Party': 'political-party-of-recipient',
    'City': 'city-of-recipient',
    'Bought out/name change' : 'did-this-company-change-its-name-or-bought-out-since-2009'
    

})

df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo = df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo[
    ['name-of-recipient-of-donation', 'name-of-donor', 'donation-amount', 'date-of-donation', 'report-year', 'political-party-of-recipient', 'city-of-recipient', 'did-this-company-change-its-name-or-bought-out-since-2009', 'original-name-of-recipient-of-donation', 'original-name-of-donor']
]

#Replace anywhere where name-of-donor is null with original-name-of-donor
df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo.loc[
    df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo['name-of-donor'].isna(), 'name-of-donor'
] = df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo['original-name-of-donor']

In [15]:
#Filter out for just candidates, parties and pacs
df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs = df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo[df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo['name-of-recipient-of-donation'].notna()]

df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs = pd.merge(df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs, df_behind_the_pacs, left_on='original-name-of-donor', right_on='Original-donor-name', how='left')


#Dropped null rows of 680 that had nothing in them
df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs = df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs.dropna(subset=['original-name-of-donor'])

#Now I need to reorganize the rows in this file
#Dropping irrevlevant columns
df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs = df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs.drop(columns=['Original-donor-name', 'Bought out/name change', 'Company-affiliated-name'  ])


In [16]:
#Filter out for just candidates, parties and pacs
df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates = df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo[df_tn_contributions_2009_to_June_30_2024_all_candidatesinfo_partiesinfo_pacsinfo_donorsinfo['name-of-recipient-of-donation'].isna()]

df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates = pd.merge(df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates, df_behind_the_pacs, left_on='original-name-of-recipient-of-donation', right_on='Original-donor-name', how='left')


#Dropped null rows of 680 that had nothing in them
df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates = df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates.dropna(subset=['original-name-of-donor'])

#Now I need to reorganize the rows in this file
#Dropping irrevlevant columns
df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates = df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates.drop(columns=['name-of-recipient-of-donation', 'name-of-donor', 'political-party-of-recipient', 'city-of-recipient', 'did-this-company-change-its-name-or-bought-out-since-2009', 'Original-donor-name'])


df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates = df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates.rename(columns={
    'Company-affiliated-name':'name-of-recipient-of-donation',
    'original-name-of-donor' : 'name-of-donor',
    'Bought out/name change' : 'did-this-company-change-its-name-or-bought-out-since-2009'
    

})

df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates = df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates[
    ['name-of-recipient-of-donation', 'name-of-donor', 'donation-amount', 'date-of-donation', 'report-year', 'did-this-company-change-its-name-or-bought-out-since-2009', 'original-name-of-recipient-of-donation']
]

#Replace anywhere where name-of-donor is null with original-name-of-donor
df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates.loc[
    df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates['name-of-recipient-of-donation'].isna(), 'name-of-recipient-of-donation'
] = df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates['original-name-of-recipient-of-donation']



In [17]:
#Currented elected status

df_current_elected_status_TN = pd.read_csv('Current-elected-status.csv')

current_elected_merged_df = pd.merge(
    df_tn_contributions_2009_to_June_30_2024_donors_to_candidates_parties_and_their_pacs,
    df_current_elected_status_TN,
    how='left', 
    on='name-of-recipient-of-donation'
)


df_tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status = current_elected_merged_df

df_tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status = df_tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status.rename(columns={
    'Current status': 'Are they currently an elected official?'
})


In [ ]:
#DATA FRAME FOR REFERENCE: Has each candidate name, party, city, results df_tn_candidates_09to24
#NOTE: Might need to figure out PAC situation and party PAC stuff
#NOTE: Probably need to get current elected status eventually

#This is all donations to candidates, parties, pacs, with current elected status
#DF1: df_tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status

#This is donations to all PACs
#DF2: df_tn_contributions_2009_to_June_30_2024_donors_to_PACs_that_donated_to_candidates

In [19]:
# This file will be used in lede-project-json-files-for-website-python-code
df_tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status.to_csv('72624-tn_contributions_2009_to_June_30_2024_all_candidates_parties_and_their_pacs_elected_status.csv', index=False)